In [1]:
import nltk
from nltk.tokenize import word_tokenize
from string import punctuation
import re
from konlpy.tag import Kkma
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

kkma = Kkma()

/Users/heeseok/anaconda3/lib/python3.7/site-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [2]:
def searchResult(_Q):
    
    searchResult = defaultdict(int)

    with open('postings.txt') as Postings_:
        for t in _Q:

            j = V.index(t)
            nextPosition = Dictionary[j]
            while True:
                Postings_.seek(nextPosition)
                _struct = [int(_.strip()) for _ in Postings_.readline().split(",")]
                #_struct[1] => 몇번나왔니
                searchResult[_struct[0]] +=1
                nextPosition = _struct[2]

                if nextPosition == -1:

                    break
        totalResult = sorted(searchResult.items(), key=lambda x:x[1], reverse=True)
    
    return totalResult

In [3]:
def select_result(totalResult):
    mx = max([_[1] for _ in totalResult])
    
    index_list=[]
    
    while sum([len(_) for _ in index_list])<=50 and mx>0:
        
        index_list.append([_[0] for _ in totalResult if _[1]==mx])
        mx -=1
        
    return index_list

In [4]:
def minwon_search(Q,data):
    _Q = [_[0] for _ in kkma.pos(Q) if _[1].startswith("N") and _[0] in V]
    totalResult = searchResult(_Q)
    _Q = ' '.join(_Q)
    print(_Q)
    index_result = select_result(totalResult)

    rankedIdx=[]
    for i in range(len(index_result)):

        temp = data.controlled_collection[index_result[i]]
        temp = np.append(temp,_Q)
        tfidf = TfidfVectorizer(vocabulary=V,sublinear_tf=True)
        tf_idf_matrix = tfidf.fit_transform(temp)
        cosine_sim = linear_kernel(tf_idf_matrix,tf_idf_matrix)
        cosine_sim = sorted(cosine_sim, key=lambda x:x[-1], reverse=True)[1:51]
        for _ in cosine_sim:
            idx = np.where(_==max(_))[0][0]
            rankedIdx.append(index_result[i][idx])

    return rankedIdx

데이터 읽어오기

In [6]:
import pandas as pd
data = pd.read_csv('total_data.csv')
len(data)

114880

lexicon 읽어오기

In [7]:
termLex = pd.read_csv('dictionary.csv')

Dictionary = defaultdict(lambda:-1)
for idx,pos in enumerate(termLex.dictionary):
    Dictionary[idx]=pos
    
V = termLex.vocabulary.to_list()

In [8]:
rankedIdx = minwon_search('건국대학교 앞 불법 노점',data)

건국 대학교 불법 노점


In [9]:
for _ in rankedIdx[:9]:
    print(re.sub(r"\r",'\n',data.iloc[_].title.strip()),'   조회수:',int(data.iloc[_]['view']))
    print('\n')
    print('---------------------------------------')
    print(re.sub(r"\r|<br\/>",'\n',data.iloc[_].question.strip()))
    print('\n')
    print('---------------------------------------')
    print(re.sub(r"\r|<br\/>",'\n',data.iloc[_].ans.strip()))
    print('=======================================')

노점상단속권한    조회수: 153


---------------------------------------
한가지 문의드립니다.
해병대전우회가 노점상 단속권한을 가지고 있는지요.
카메라를 들고 다니면서 노점상을 단속한다고 하더군요.장소는 건국대학교 근처입니다.
답변부탁드립니다.


---------------------------------------
구정에 대하여 적극적 관심에 감사를 드립니다.2008. 2. 25일부터 건대역주변에서 아차산역에 이르는 구간에 대하여 주민을 위한 통행권 확보 및 도시미관 향상을 위하여 노점 환경정비는 물론 신발생노점에 대하여 계도하도록 고용되어 있으며, 롯데백화점예정지 및 건대병원 주변 보도에 노점이 발생치 않토록 예방 목적으로 일하고 있음을 알려드립니다.가내 행복이 깃들기를 기원합니다. 감사합니다.
낙원상가 옆 쓰레기 무단투기 및 불법주차    조회수: 235


---------------------------------------
안녕하세요
 프레이저 스위츠 호텔 (낙원동 272번지) 보안팀 입니다. 
 다름이 아니라 호텔 주변에 쓰레기 투기 및 불법 주정차, 야간에는 노상방뇨 등으로 골치를 앓고 있습니다.
 가장 심각한것은 쓰레기 투기와 불법주차 입니다. 
 불법주차의 경우에는 호텔 앞,뒤로 상당히 심합니다. 이로 인해 오후에는 호텔 주변 도로가 마비상태가
 되어버립니다. 건국주차장과 호텔 사이로 있는 도로 (낙원상가에서 인사동 네거리 방향)에 불법주차는 
 단속오시면 알겠지만 365일 24시간 무조건 4대~8대가 있습니다. 주말에는 불법주차가 양방향으로
 들어서서 통제불능입니다. 호텔 뒷편 (커피빈방향)에도 4대가량이 항상 있습니다. 
 불법주차 차량들의 단속이 없는것을 보고, 현재 우리 유료주차장을 이요하는 고객 또한 정기권 주차를
 포기하고 똑같이 불법주차를 하는 상황입니다. 이는 우리호텔에 입주하여 있는 위탁운영 업체의 
 수익문제와도 직결되며, 호텔 이미지 훼손에도 큰 영향을 미치고 있습니다. 
 쓰레기 투

In [42]:
from konlpy.tag import Hannanum
kkma = Hannanum()

In [44]:
#문제있다
print(kkma.nouns('안암역 불법노점 '))
print(kkma.nouns('안암오거리 불법노점 '))
print()


print(kkma.pos('문정역 불법노점 '))
print(kkma.pos('문정동 불법노점 '))
print()


print(kkma.pos('장지역 불법노점 '))
print(kkma.pos('장지동 불법노점 '))
print()


print(kkma.pos('건대앞 불법노점 '))  # 줄임말 안쓰면 가능..
print(kkma.pos('건대입구역 앞 불법노점 '))
print(kkma.pos('건국대학교 앞 불법노점 '))
print()


print(kkma.pos('종로3가 불법노점 '))

['안암역', '불법노점']
['안암오거리', '불법노점']

[('문정역', 'N'), ('불법노점', 'N')]
[('문정동', 'N'), ('불법노점', 'N')]

[('장지역', 'N'), ('불법노점', 'N')]
[('장지동', 'N'), ('불법노점', 'N')]

[('건대앞', 'N'), ('불법노점', 'N')]
[('건대입구역', 'N'), ('앞', 'N'), ('불법노점', 'N')]
[('건국대학교', 'N'), ('앞', 'N'), ('불법노점', 'N')]

[('종로3', 'N'), ('가', 'J'), ('불법노점', 'N')]


In [32]:
data = data.dropna()
tfidf = TfidfVectorizer(vocabulary=V,sublinear_tf=True)
tf_idf_matrix = tfidf.fit_transform(data.controlled_collection)

In [39]:
tfidf

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=True, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True,
                vocabulary=['서울시', '단기', '보호', '시설', '근무', '종사자', '입사', '이래',
                            '환경', '조건', '열심', '상충', '불합리', '정책', '처우', '근무자',
                            '상황', '분통', '호소', '현재', '중앙', '환원', '기와', '그룹',
                            '제외', '2015', '임금', '테이블', '제기', '시장', ...])

In [33]:
tf_idf_matrix

<114876x82914 sparse matrix of type '<class 'numpy.float64'>'
	with 9133814 stored elements in Compressed Sparse Row format>

In [34]:
tfidf
tf_idf_matrix.toarray()[0]

array([0.13575472, 0.27959777, 0.06124245, ..., 0.        , 0.        ,
       0.        ])

In [27]:
data.tail(2)

,site_no,site_index,title,date,view,question,part,ans_date,ans,collection,controlled_collection
114878,0,443344,식품 원료 가능 여부(딸기 씨앗),2018.08.29,2.0,딸기의 씨앗을 식품원료로 사용할 수 있나요?,식품의약품안전처 기획조정관 고객지원담당관 (☎ 1577-1255),2018.08.29,\n\n1. 안녕하십니까? 우리처 국민신문고(사이버상담)를 방문하여 주셔서 감사합니...,딸기의 씨앗을 식품원료로 사용할있나요 1. 안녕하십니까국민신문고 사이버방문하여 주셔...,딸기 씨앗 식품 원료 사용 안녕 신문고 사이버 방문 질의 내용 아래 기가 식품 기준...
114879,0,443250,식품 원료 가능 여부(이성화액당),2018.08.29,1.0,이성화액당(Isomerized glucose syrup)을 식품의 원료로 사용할 수...,식품의약품안전처 기획조정관 고객지원담당관 (☎ 1577-1255),2018.08.29,\n\n1. 안녕하십니까? 우리처 국민신문고(사이버상담)를 방문하여 주셔서 감사합니...,이성화액당식품의 원료로 사용할있나요 1. 안녕하십니까국민신문고 사이버방문하여 주셔서...,이성화 식품 원료 사용 안녕 신문고 사이버 방문 질의 내용 아래 이성화 전분 주원료...
